# ASEを用いた分子シミュレーション  (M3GNetの場合)
<BR>

参考文献（M3GNet）<BR>
- GITHUB https://github.com/materialsvirtuallab/m3gnet
- Chen, C., Ong, S.P. A universal graph deep learning interatomic potential for the periodic table. Nat Comput Sci 2, 718–728 (2022). https://doi.org/10.1038/s43588-022-00349-3. (論文引用先)

  
作成：中山将伸 (2023.11.1)  再配布禁止<BR>

## 【必須】ASEモジュール他のインポート


In [1]:
import numpy as np
import math, random

import os,sys,csv,glob,shutil,re,time
from time import perf_counter
from joblib import Parallel, delayed
args = sys.argv

# sklearn
# from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

import ase #
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, UnitCellFilter
from ase.optimize import LBFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase import Atoms
from ase.io import read, write
from ase.io import Trajectory
from ase import units
from ase.build import bulk, make_supercell
from ase.visualize import view


## 【必須】Atoms object 入力

In [2]:
#結晶構造ファイルの読み込み

inpf1="./inputs/LiFePO4.cif"   #　脱リチウム前（放電後）の構造
inpf2="./inputs/FePO4.cif"     #  脱リチウム後（充電後）の構造
inpf3="./inputs/Li.cif"        #  リチウム金属
guestion="Li"           #　ゲストイオン種
charge_gi=1             #  ゲストイオンの電荷

atoms1=read(inpf1)
atoms2=read(inpf2)
atoms3=read(inpf3)

num_gi1=atoms1.get_chemical_symbols().count(guestion)
num_gi2=atoms2.get_chemical_symbols().count(guestion)
num_gi3=atoms3.get_chemical_symbols().count(guestion)


print ("composition:", atoms1.symbols, atoms2.symbols, atoms3.symbols)
print ("number of guest ions:", num_gi1, num_gi2, num_gi3)


composition: Li4Fe4P4O16 Fe4P4O16 Li2
number of guest ions: 4 0 2


## 【必須】Atoms objectにM3GNetのポテンシャルを導入

In [3]:
#from m3gnet.models import M3GNet, M3GNetCalculator, Potential
#potential = Potential(M3GNet.load()) #M3GNetのデフォルトポテンシャルを導入　（自分で作ったポテンシャルは要別途指定）
#calculator = M3GNetCalculator(potential=potential, stress_weight=0.01)

# This is code for mgl
import matgl
from matgl.ext.ase import M3GNetCalculator
potential = matgl.load_model("M3GNet-MP-2021.2.8-PES")
calculator = matgl.ext.ase.M3GNetCalculator(potential=potential, stress_weight=0.01)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone(

## M3GNet 構造緩和計算

各種アルゴリズムがある(FIRE, LBFGSなど　下記の例はLBFGSを使用)<BR>
得られたエネルギー値から電位などを計算することもできる。<BR>
NVT-MDを行う前に一度構造緩和計算をすることを推奨（格子定数を適切な値にするため）<BR>

In [4]:
#relax 計算
maxstep=2000

from ase.optimize import LBFGS, BFGS, FIRE
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, UnitCellFilter, ExpCellFilter

# 
#atoms1.set_constraint()
#atoms1.set_calculator(calculator)
atoms1.calc=calculator
ucf=ExpCellFilter(atoms1)
opt = LBFGS(ucf, trajectory="relax1.traj",logfile="log.relax1")
opt.run(steps=maxstep)                    
energy1=atoms1.get_potential_energy()
force1=atoms1.get_forces()
t=read('relax1.traj',index=':')
nt=len(t)
if nt== maxstep:
    crt=False
else:
    crt=True
print ("Input-file 1: (energy)", energy1,"/eV,   relaxation steps", nt, ",  converged:", crt)

#atoms2.set_constraint()
#atoms2.set_calculator(calculator)
atoms2.calc=calculator
ucf=ExpCellFilter(atoms2)
opt = LBFGS(ucf, trajectory="relax2.traj",logfile="log.relax2")
opt.run(steps=maxstep)                    
energy2=atoms2.get_potential_energy()
force2=atoms2.get_forces()
t=read('relax2.traj',index=':')
nt=len(t)
if nt== maxstep:
    crt=False
else:
    crt=True
print ("Input-file 2: (energy)", energy2,"/eV,   relaxation steps", nt, ",  converged:", crt)

#atoms3.set_constraint()
#atoms3.set_calculator(calculator)
atoms3.calc=calculator
ucf=ExpCellFilter(atoms3)
opt = LBFGS(ucf, trajectory="relax3.traj",logfile="log.relax3")
opt.run(steps=maxstep)                    
energy3=atoms3.get_potential_energy()
force3=atoms3.get_forces()
t=read('relax3.traj',index=':')
nt=len(t)
if nt== maxstep:
    crt=False
else:
    crt=True
print ("Input-file 3: (energy)", energy3,"/eV,   relaxation steps", nt, ",  converged:", crt)


# print ("energy:",energy)
# print ("cell parameters:", atoms.get_cell_lengths_and_angles())
# maxf = np.sqrt(((force**2).sum(axis=1).max()))
# print ("max force",maxf)

# #ase.io.write("CONTCAR", atoms, format="vasp")
# ase.io.write("relax.cif", atoms, format="cif")



/var/folders/0f/gst3ndcj1gl6tx25nxpl4vh80000gn/T/ipykernel_77736/3295633956.py:11: FutureWarning: Import ExpCellFilter from ase.filters
  ucf=ExpCellFilter(atoms1)


Input-file 1: (energy) -190.59022521972656 /eV,   relaxation steps 24 ,  converged: True


/var/folders/0f/gst3ndcj1gl6tx25nxpl4vh80000gn/T/ipykernel_77736/3295633956.py:27: FutureWarning: Import ExpCellFilter from ase.filters
  ucf=ExpCellFilter(atoms2)


Input-file 2: (energy) -169.539794921875 /eV,   relaxation steps 53 ,  converged: True
Input-file 3: (energy) -3.8039050102233887 /eV,   relaxation steps 2 ,  converged: True


/var/folders/0f/gst3ndcj1gl6tx25nxpl4vh80000gn/T/ipykernel_77736/3295633956.py:43: FutureWarning: Import ExpCellFilter from ase.filters
  ucf=ExpCellFilter(atoms3)


## 電位計算

オリビンの場合：  
反応式  Li + FePO4 --> LiFePO4  (反応に関与する電子数 n=1)

Voltage =  - [ E(LiFePO4) - { n*E(Li) + E(FePO4) }] /n 

In [5]:
#電位計算

voltage=(energy2+(num_gi1-num_gi2)/(num_gi3)*energy3-energy1)/((num_gi1-num_gi2)*charge_gi)

print ("voltage:", voltage, " /V")

voltage: 3.3606550693511963  /V


In [ ]:
#ファイル出力（構造緩和後）
write("relax_structure1.cif",atoms1,format="cif")
write("relax_structure2.cif",atoms2,format="cif")
write("relax_structure3.cif",atoms3,format="cif")


In [8]:
#maskの設定 結果
#mask = [[1,1,1], [1,1,1], [1,1,1]] #全方向・全成分緩和（デフォルト）
mask = [[1,0,0], [0,0,0], [0,0,0]] #α軸方向のセルのみ変形OK
#mask = [[0,0,0], [0,0,0], [0,0,0]] #全セル固定（セル変形なし、原子だけ緩和）
#mask = [[1,1,0], [1,1,0], [0,0,0]] #ab面だけ変形OK, c軸は固定

atoms1 = read(inpf1)
print(atoms1.get_cell())
atoms1.calc=calculator

# ExpCellFilter に mask を設定
ecf = ExpCellFilter(atoms1, mask=mask)

#最適化
opt = BFGS(ecf)
opt.run(fmax=0.01)

atoms1.get_cell()

Cell([4.65491719, 5.9707551, 10.23619605])


/var/folders/0f/gst3ndcj1gl6tx25nxpl4vh80000gn/T/ipykernel_77736/3474069008.py:12: FutureWarning: Import ExpCellFilter from ase.filters
  ecf = ExpCellFilter(atoms1, mask=mask)


      Step     Time          Energy          fmax
BFGS:    0 17:00:00     -190.180679       26.307216
BFGS:    1 17:00:01     -185.667679       80.506578
BFGS:    2 17:00:01     -190.327667       14.739340
BFGS:    3 17:00:01     -190.292923       23.540860
BFGS:    4 17:00:02     -190.463333        1.630482
BFGS:    5 17:00:02     -190.475525        0.306142
BFGS:    6 17:00:02     -190.487732        3.147047
BFGS:    7 17:00:02     -190.493637        2.715340
BFGS:    8 17:00:03     -190.504608        1.169696
BFGS:    9 17:00:03     -190.508362        1.360683
BFGS:   10 17:00:03     -190.512177        0.438127
BFGS:   11 17:00:04     -190.514328        0.201442
BFGS:   12 17:00:04     -190.517609        0.260838
BFGS:   13 17:00:04     -190.518402        0.220431
BFGS:   14 17:00:05     -190.523285        0.097682
BFGS:   15 17:00:05     -190.523697        0.057871
BFGS:   16 17:00:05     -190.524445        0.066984
BFGS:   17 17:00:06     -190.525589        0.157135
BFGS:   18 17:

Cell([4.869549132611826, 5.9707551, 10.23619605])